In [1]:
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from urllib.parse import urlencode, quote_plus
from datetime import datetime, date, timedelta
import os
import json

FMP_KEY = os.getenv("FMP_KEY")
FMP_URL = 'https://financialmodelingprep.com'

TICKER = "^GSPC"
enddt = datetime.today()                      # last day for price series
startdt = enddt + timedelta(days=-100*365.25) # Give me the last hundred years
ENDDT = enddt.isoformat()[:10]
STARTDT = startdt.isoformat()[:10]

In [2]:
def get_quoteHistory(ticker, startdt=STARTDT, enddt=ENDDT):
    """
    Obtain a dataframe with historical asset price timeseries
    for a given ticker
    ----------
    ticker : str
    startdt: str (YYYY-MM-DD)
    enddt  : str (YYYY-MM-DD)
    
    Returns
    -------
    pandas dataframe
    """
    payload = urlencode(
        {"apikey": FMP_KEY,
         "from"  : startdt,
         "to"    : enddt}
    )
    tickerEnc = quote_plus(ticker)
    endpoint_qh = f"/api/v3/historical-price-full/{tickerEnc}?"
    url = FMP_URL + endpoint_qh + payload
    response = urlopen(url)
    data = json.loads(response.read().decode("utf-8"))
    return pd.DataFrame.from_dict(data['historical'])

In [3]:
spxprices = get_quoteHistory(TICKER)
spxprices['date'] = pd.to_datetime(spxprices['date'])
spxprices['yrq'] = spxprices.date.dt.to_period("Q")
spxprices['year'] = spxprices.date.dt.year
spxprices['qtr'] = spxprices.date.dt.quarter
spxprices = spxprices.loc[spxprices.yrq < spxprices.yrq.max(),
                          ["date","yrq","year","qtr","adjClose"]]
spxprices = spxprices.loc[:,["date","yrq","year","qtr","adjClose"]]
spxprices.set_index("date", drop=True, inplace=True)
spxprices.columns = ['yrq','year','qtr', 'spx']
spxprices['ytdRet']  = spxprices.spx / spxprices['spx'].groupby(spxprices.index.year).transform('last') - 1
spxprices

,yrq,year,qtr,spx,ytdRet
date,,,,,
2022-09-30,2022Q3,2022,3,3585.620100,-0.252460
2022-09-29,2022Q3,2022,3,3640.470000,-0.241025
2022-09-28,2022Q3,2022,3,3719.040000,-0.224644
2022-09-27,2022Q3,2022,3,3647.290000,-0.239603
2022-09-26,2022Q3,2022,3,3655.040000,-0.237987
...,...,...,...,...,...
1928-01-06,1928Q1,1928,1,17.660000,-0.005631
1928-01-05,1928Q1,1928,1,17.549999,-0.011824
1928-01-04,1928Q1,1928,1,17.719999,-0.002252


In [5]:
qs = spxprices.groupby("yrq").first()
qs['nextQRet'] = qs.spx.shift(-1)/qs.spx - 1
qs.to_csv("4Q SPX perf.csv")

In [6]:
qs

,year,qtr,spx,ytdRet,nextQRet
yrq,,,,,
1927Q4,1927,4,17.660000,0.000000,0.091733
1928Q1,1928,1,19.280001,0.085586,-0.007262
1928Q2,1928,2,19.139999,0.077703,0.104493
1928Q3,1928,3,21.139999,0.190315,0.151845
1928Q4,1928,4,24.350000,0.371059,0.048460
...,...,...,...,...,...
2021Q3,2021,3,4307.540039,0.163996,0.106474
2021Q4,2021,4,4766.180176,0.287931,-0.049467
2022Q1,2022,1,4530.410156,-0.055488,-0.164451
